# Converting the CSV files to FITS files that can be read by FastSpecFit

FastSpecFit is currently only able to ingest spectra that are on as wavelength grid with equal wavelength bins. So aside from just reformatting the files I also need to resample all the spectra. Currently this is done using a linear interpolation on a new wavelength grid.

In [57]:
import glob
import numpy as np
from astropy.table import Table
import fitsio

sys.path.insert(0,'/global/u2/d/dscholte/forked_packages/desigal/py')
import desigal.specutils
from desigal.specutils.resample import linear_resample

In [29]:
spectra_files = glob.glob('Spectra*.csv')

In [105]:
int((len(table.colnames)-3)/2)

3

In [158]:
def convert_spectra_file(filename, new_wave):
    table = Table.read(filename)
    nobj = int((len(table.colnames)-3)/2)
    flux = np.zeros((nobj, len(new_wave)))
    ivar = np.zeros((nobj, len(new_wave)))
    for i in range(nobj):
        fl, iv = linear_resample(
            new_wave, 
            np.expand_dims(table['Wavelength'].data, axis=0), 
            np.expand_dims(table['flux_radius_'+str(i+1)].data, axis=0), 
            ivar=np.expand_dims(table['ivar_radius_'+str(i+1)].data, axis=0)
        )
        flux[i] = fl[0]
        ivar[i] = iv[0]
    mask = np.isnan(flux)|np.isnan(ivar)
    flux[mask] = 0.0
    ivar[mask] = 0.0

    desigal.specutils.stack.write_binned_stacks(
        filename[:-3]+'fits', 
        new_wave,
        flux,
        ivar,
        stackids = np.arange(1,nobj+1),
        table_column_dict = {},
        table_format_dict = {},
    )
    return

In [159]:
new_wave = np.arange(3550,9500,1) # 1 Angstrom wavelength bins
[convert_spectra_file(filename, new_wave) for filename in spectra_files]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]